In [13]:
import cv2
import numpy as np
import time

In [14]:
color_ranges = {
    'red': ([0, 120, 70], [10, 255, 255]),
    'green': ([36, 25, 25], [86, 255, 255]),
    'yellow': ([25, 100, 100], [35, 255, 255]),
    'white': ([0, 0, 200], [180, 20, 255]),
}

In [15]:
def detect_balls(frame, color_ranges):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    balls = {}
    for color, (lower, upper) in color_ranges.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)
        mask = cv2.inRange(hsv, lower, upper)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)
        cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if cnts:
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            if radius > 10:
                balls[color] = (int(x), int(y), int(radius))
    return balls

In [16]:
def determine_quadrant(x, y, width, height):
    if x < width / 2:
        if y < height / 2:
            return 1
        else:
            return 3
    else:
        if y < height / 2:
            return 2
        else:
            return 4

In [17]:
cap = cv2.VideoCapture('AI Assignment video.mp4')

In [18]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [19]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('processed_video.avi', fourcc, fps, (width, height))

In [20]:
ball_positions = {}
events = []

In [21]:
start_time = time.time()
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    balls = detect_balls(frame, color_ranges)

    for color, (x, y, radius) in balls.items():
        quadrant = determine_quadrant(x, y, width, height)

        if color not in ball_positions:
            ball_positions[color] = None

        previous_quadrant = ball_positions[color]
        ball_positions[color] = quadrant

        if previous_quadrant is None:
            # First detection, assume entry
            event_type = 'Entry'
        elif previous_quadrant != quadrant:
            event_type = 'Entry' if previous_quadrant is None else 'Exit'
            events.append((frame_num / fps, quadrant, color, event_type))
            event_text = f"{event_type}: Q{quadrant} at {frame_num / fps:.2f}s"
            cv2.putText(frame, event_text, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            cv2.putText(frame, event_text, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Draw the ball and the quadrant on the frame
        cv2.circle(frame, (x, y), radius, (0, 255, 255), 2)
        cv2.putText(frame, f"{color}", (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        cv2.putText(frame, f"{color}", (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Draw quadrant lines
    cv2.line(frame, (width // 2, 0), (width // 2, height), (255, 255, 255), 2)
    cv2.line(frame, (0, height // 2), (width, height // 2), (255, 255, 255), 2)

    out.write(frame)
    frame_num += 1


In [22]:
cap.release()
out.release()

In [23]:
with open('event_log.txt', 'w') as f:
    for event in events:
        f.write(f"{event[0]:.2f}, {event[1]}, {event[2]}, {event[3]}\n")


In [24]:
print("Processing complete. Processed video saved as 'processed_video.avi' and event log saved as 'event_log.txt'.")

Processing complete. Processed video saved as 'processed_video.avi' and event log saved as 'event_log.txt'.
